In [13]:

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt

In [14]:
df = pd.read_csv('data/areaverde.csv')
df['Localidade'] = df['Localidade'].str.upper()
df = df.set_index('Localidade')

df2 = pd.read_csv('data/populacao.csv', index_col=False)
df2['Localidade'] = df2['Localidade'].str.upper()

df3 = pd.read_csv('data/renda.csv', index_col=False)
df3['Localidade'] = df3['Localidade'].str.upper()

df4 = pd.read_csv('data/covid.csv', index_col=False)
df5 = pd.read_csv('data/dengue.csv', index_col=False)
df6 = pd.read_csv('data/num_atendimentos.csv', index_col=False)
df7 = pd.read_csv('data/num_unidades_saude.csv', index_col=False)
df7['Localidade'] = df7['Localidade'].str.upper()


df = df.merge(df2, on='Localidade', how='left')
df = df.merge(df3, on='Localidade', how='left')

df = df.merge(df4, on='Localidade', how='left')
df = df.merge(df5, on='Localidade', how='left')
df = df.merge(df6, on='Localidade', how='left')
df = df.merge(df7, on='Localidade', how='left', suffixes=('_x1', '_x2'))

df['Localidade'] = df['Localidade'].str.upper()

# df['porcentagem_cobertura'] = df['area_verde'] / df['area']

df['porcentagem_brancos'] = df['popBrancos'] / df['popTotal']
df['porcentagem_adultos'] = df['pop15a64'] / df['popTotal']

df['taxa_obitos_covid'] = df['num_obitos_covid'] / df['popTotal']
df['taxa_atendimentos_covid'] = df['num_atendimentos_covid'] / df['popTotal']
df['taxa_atendimentos_dengue'] = df['num_atendimentos_dengue'] / df['popTotal']
df['taxa_atendimentos_saude'] = df['num_atendimentos_saude_ja'] / df['popTotal']
df['taxa_unidades_saude'] = df['num_unidades_saude_bairro'] / df['popTotal']

df = df.fillna(0)
df


,Localidade,Ano_x,areasVerdesM2,areasVerdesM2porHab,Ano_y,popTotal,popHomens,popBrancos,popTotal2,popHabPHa,...,num_atendimentos_saude_ja,Unnamed: 0_x2,num_unidades_saude_bairro,porcentagem_brancos,porcentagem_adultos,taxa_obitos_covid,taxa_atendimentos_covid,taxa_atendimentos_dengue,taxa_atendimentos_saude,taxa_unidades_saude
0,ABRANCHES,2019,"1622321,89","105,23",2010,13189,6434,10214,13189,"30,56",...,8727.0,0.0,2.0,0.774433,0.710668,0.004398,0.334445,0.003564,0.661688,0.000152
1,ÁGUA VERDE,2019,"807292,36","15,19",2010,51425,23228,46875,51425,"107,74",...,0.0,62.0,10.0,0.911522,0.757355,0.004064,0.267963,0.002450,0.000000,0.000194
2,AHÚ,2019,"370152,07","31,1",2010,11506,5266,10550,11506,"62,03",...,0.0,1.0,1.0,0.916913,0.722666,0.003824,0.286807,0.001912,0.000000,0.000087
3,ALTO BOQUEIRÃO,2019,"3610320,14","63,97",2010,53671,25832,40918,53671,"44,77",...,0.0,2.0,4.0,0.762386,0.721507,0.004528,0.331333,0.016079,0.000000,0.000075
4,ALTO DA GLÓRIA,2019,"130283,27","23,67",2010,5548,2493,5113,5548,"63,37",...,0.0,3.0,3.0,0.921593,0.750721,0.007030,0.416907,0.004326,0.000000,0.000541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
70,UBERABA,2019,"2008199,25","23,64",2010,72056,34959,54586,72056,"50,71",...,51282.0,57.0,6.0,0.757550,0.712113,0.004746,0.360248,0.005371,0.711696,0.000083
71,UMBARÁ,2019,"9265761,46","397,98",2010,18730,9270,14303,18730,"8,34",...,0.0,58.0,2.0,0.763641,0.699092,0.004805,0.381367,0.003257,0.000000,0.000107
72,VILA IZABEL,2019,"200094,92","16,22",2010,11610,5269,10555,11610,"95,44",...,3243.0,59.0,3.0,0.909130,0.761068,0.005340,0.405857,0.003359,0.279328,0.000258
73,VISTA ALEGRE,2019,"1231472,42","97,77",2010,11199,5290,9689,11199,"30,28",...,5796.0,60.0,1.0,0.865167,0.709081,0.004108,0.345031,0.003215,0.517546,0.000089


In [15]:
import ipywidgets as widgets
from IPython.display import display


gdf = gpd.read_file('DIVISA_DE_BAIRROS/DIVISA_DE_BAIRROS.shp')
gdf = gdf.merge(df, left_on='NOME', right_on='Localidade', how='right')
gdf['area'] = gdf.area
gdf['area_verde'] = gdf['areasVerdesM2'].str.replace(',', '.').astype(float)

# Convert numeric columns from string to float, handling comma decimal separator
numeric_columns = gdf.select_dtypes(include=['object']).columns
for col in numeric_columns:
    try:
        # Try converting comma decimal separator to period and convert to float
        gdf[col] = gdf[col].str.replace(',', '.').astype(float)
    except (AttributeError, ValueError):
        # Skip if column can't be converted to float
        continue


w = widgets.Dropdown(
    options=list(gdf),
    value='Localidade',
    description='Data:',
)

map_output = widgets.Output()

display(w, map_output)

Dropdown(description='Data:', index=10, options=('OBJECTID', 'CODIGO', 'TIPO', 'NOME', 'FONTE', 'CD_REGIONA', …

Output()

In [16]:
from IPython.display import clear_output

def update_map(change):
    with map_output:
        clear_output(wait=True)
        display(gdf.explore(column=change.new, legend=True, legend_kwds={"label": change.new, "orientation": "vertical"}))

w.observe(update_map, names='value')

# Initial map display
with map_output:
    display(gdf.explore(column=w.value, legend=True, legend_kwds={"label": w.value, "orientation": "vertical"}))

In [17]:
# Save the GeoDataFrame to a GeoJSON file
gdf.to_file("curitiba_bairros_completo.geojson", driver='GeoJSON')
